In [ ]:
import sys
fileDir = "/home/jovyan/notebooks/"
sys.path.append(fileDir)

from utilities import *

import pyspark.sql.functions as F
import pyspark.sql.types as T

In [ ]:
session = create_spark_session("Neo4j GitHub", SparkConnector.NEO4J)

In [ ]:
# Scenario 1

options = get_default_options(SparkConnector.NEO4J)
options["query"] =  """
                        MATCH (contrib:GitContributor)-[:AUTHOR]->(commit:GitCommit)-[:BELONGS_TO]->(repo:GitRepository)
                        WITH contrib, COUNT(DISTINCT repo) as repo_count
                        RETURN contrib, repo_count ORDER BY repo_count DESC
                    """
top10contributors = spark_read(SparkConnector.NEO4J, session, options=options)
display(top10contributors.take(10))

In [ ]:
# Scenario 2:

LANGUAGE = "C++"
PERCENTAGE = 0.5
options = get_default_options(SparkConnector.NEO4J)
options["query"] = f"""
                    MATCH (r:GitRepository)-[w:WRITTED_IN]->(l:GitLanguage)
                    WITH r, SUM(w.bytes) AS totalBytesForRepo, collect({{language_name:l.name,bytes: w.bytes}}) AS bytesForLanguages
                    UNWIND bytesForLanguages AS bytesForLanguage
                    WITH r.name AS repo_name, bytesForLanguage.language_name AS lang, round((bytesForLanguage.bytes*1.0/totalBytesForRepo),2) AS percOfBytes
                        WHERE lang = "{LANGUAGE}" AND percOfBytes > {PERCENTAGE}
                    RETURN repo_name, lang, percOfBytes 
                  """
bytesPercentageInRepos = spark_read(SparkConnector.NEO4J, session, options=options)
display(bytesPercentageInRepos.take(10))

In [ ]:
# Scenario 3:
REPO_NAME = "tensorflow/tensorflow"
options = get_default_options(SparkConnector.NEO4J)
options["query"] =  f"""
                        MATCH (repository:GitRepository {{name: "{REPO_NAME}"}})<-[:BELONGS_TO]-(commit:GitCommit), 
                            r = (commit)-[:PARENT]->()
                        WITH commit, collect(r) AS parents
                        WHERE size(parents) > 1
                        RETURN count(commit) AS mergeCount
                    """
bytesPercentageInRepos = spark_read(SparkConnector.NEO4J, session, options=options)
bytesPercentageInRepos.show(10)

In [ ]:
# Scenario 4 - project graph repo and commits:
query = """
             CALL gds.graph.project.cypher(
              'repoAndCommits',
              'MATCH (n:GitCommit) RETURN ID(n) AS id 
              UNION 
              MATCH (n:GitRepository) RETURN ID(n) as id',
              'MATCH (n:GitCommit)-[:PARENT]->(m:GitCommit) RETURN ID(n) AS source, ID(m) AS target 
              UNION 
              MATCH (c:GitCommit)-[:BELONGS_TO]->(r:GitRepository) RETURN ID(c) AS source, ID(r) AS target'
                )
        """
raise Exception("Put query text in the neo4j GUI")

In [ ]:
# Scenario 4 - project graph contributors, commits and repos:
query = """
             CALL gds.graph.project.cypher(
              'contribRepoAndCommits',
              'MATCH (n:GitContributor) RETURN ID(n) AS id 
               UNION 
               MATCH (n:GitCommit) RETURN ID(n) as id
               UNION
               MATCH (n:GitRepository) RETURN ID(n) as id',
              'MATCH (u:GitContributor)-[:COMMITTED]->(w:GitCommit) RETURN ID(u) AS Source, ID(w) AS Target 
               UNION
               MATCH (s:GitContributor)-[:AUTHOR]->(t:GitCommit) RETURN ID(s) AS Source, ID(t) AS Target 
               UNION
               MATCH (n:GitCommit)-[:PARENT]->(m:GitCommit) RETURN ID(n) AS Source, ID(m) AS Target 
               UNION 
               MATCH (c:GitCommit)-[:BELONGS_TO]->(r:GitRepository) RETURN ID(c) AS Source, ID(r) AS Target'
        )
        """
raise Exception("Put query text in the neo4j GUI")


In [ ]:
# Scenario 4 - Louvain:
PROJ_NAME = "repoAndCommits" or "contribRepoAndCommits"

options = get_default_options(SparkConnector.NEO4J)
options["query"] =  f"""
                        CALL gds.louvain.stream('{PROJ_NAME}')
                        YIELD nodeId, communityId, intermediateCommunityIds
                        RETURN nodeId as ID, communityId 
                    """
louvain = spark_read(SparkConnector.NEO4J, session, options=options)
louvain.write.option("header", True).mode("overwrite").csv("hdfs://namenode:9000//data-team/louvain")

In [ ]:
# Scenario 4 - WCC:
options = get_default_options(SparkConnector.NEO4J)
options["query"] =  f"""
                        CALL gds.wcc.stream('{PROJ_NAME}') YIELD nodeId, componentId
                        RETURN nodeId AS ID, componentId
                    """
wcc = spark_read(SparkConnector.NEO4J, session, options=options)
wcc.write.option("header", True).mode("overwrite").csv("hdfs://namenode:9000//data-team/wcc")

In [ ]:
session.sparkContext.stop()
session.stop()